# Продолжнеие работы

## Прогнозирование лайка/дизлайка

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
path = 'drive/MyDrive/DS/meme'

In [ ]:
df = pd.read_csv(path+'/splitted_reactions_df10.csv', index_col = 0)

In [ ]:
df

,event_id,reaction_id,meme_id,user_id,published_at2021,content_lang,is_available,likes,dislikes,source_id,content_text,language_id,meme_type_animation,meme_type_photo,meme_type_video,source_grouped,clusters_meme,sessions,last_session_memes,watched_memes,interface_lang,clusters_user
0,1569409,1.0,9197,1153946021,0,0,1,554,600,46.0,1,2.0,0,1,0,0,1,39,1,636,0,0
1,1563971,1.0,46731,1153946021,0,0,1,1660,1154,8.0,1,2.0,0,1,0,0,1,39,1,636,0,0
2,1564013,1.0,49571,1153946021,0,0,1,839,381,21.0,1,2.0,0,1,0,0,1,39,1,636,0,0
3,1569414,1.0,53977,1153946021,0,0,1,564,260,21.0,1,2.0,0,1,0,0,1,39,1,636,0,0
4,1569391,1.0,58789,1153946021,0,0,1,466,212,21.0,1,2.0,0,1,0,0,1,39,1,636,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716650,4242453,2.0,23735,281359417,0,0,1,166,178,50.0,1,2.0,0,1,0,0,1,22,1,699,0,0
716651,5286957,2.0,46075,281359417,0,0,1,1189,587,18.0,1,2.0,0,1,0,0,1,22,1,699,0,0
716652,5288899,2.0,26207,281359417,0,0,1,971,442,21.0,1,2.0,0,1,0,0,1,22,1,699,0,0
716653,5288058,2.0,60707,281359417,0,0,1,969,878,46.0,1,2.0,0,1,0,0,1,22,1,699,0,0


In [ ]:
df.isna().sum()

event_id               0
reaction_id            0
meme_id                0
user_id                0
published_at2021       0
content_lang           0
is_available           0
likes                  0
dislikes               0
source_id              0
content_text           0
language_id            0
meme_type_animation    0
meme_type_photo        0
meme_type_video        0
source_grouped         0
clusters_meme          0
sessions               0
last_session_memes     0
watched_memes          0
interface_lang         0
clusters_user          0
dtype: int64

In [ ]:
df.drop(columns = ['event_id', 'user_id', 'meme_id'], axis =1 , inplace = True)

In [ ]:
X = df.iloc[:,1:]

In [ ]:
y = df.iloc[:,0:1]

In [ ]:
y

,reaction_id
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
716650,2.0
716651,2.0
716652,2.0
716653,2.0


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2 , random_state = 1)

In [ ]:
# data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(X_train)

# transform training data
X_train_norm = norm.transform(X_train)

# transform testing dataabs
X_test_norm = norm.transform(X_test)

In [ ]:
# data standardization with  sklearn
from sklearn.preprocessing import StandardScaler

# copy of datasets
X_train_stand = X_train.copy()
X_test_stand = X_test.copy()

# numerical features
num_cols = ['likes','dislikes','sessions','watched_memes']

# apply standardization on numerical features
for i in num_cols:
    
    # fit on training data column
    scale = StandardScaler().fit(X_train_stand[[i]])
    
    # transform the training data column
    X_train_stand[i] = scale.transform(X_train_stand[[i]])
    
    # transform the testing data column
    X_test_stand[i] = scale.transform(X_test_stand[[i]])

In [ ]:
# training a KNN model
from sklearn.neighbors import KNeighborsClassifier
# measuring RMSE score
from sklearn.metrics import mean_squared_error

# knn 
knn = KNeighborsClassifier(n_neighbors=7)

rmse = []

# raw, normalized and standardized training and testing data
trainX = [X_train, X_train_norm, X_train_stand]
testX = [X_test, X_test_norm, X_test_stand]
pred = []

# model fitting and measuring RMSE
for i in range(len(trainX)):
    
    # fit
    knn.fit(trainX[i],y_train)
    # predict
    pred.append(knn.predict(testX[i]))
    # RMSE
    rmse.append(np.sqrt(mean_squared_error(y_test,pred[i])))

# visualizing the result
#df_knn = pd.DataFrame({'RMSE':rmse},index=['Original','Normalized','Standardized'])
#df_knn

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


KeyboardInterrupt: ignored

In [ ]:
#f_knn = pd.DataFrame({'RMSE':rmse},index=['Original','Normalized','Standardized'])
#df_knn

In [ ]:
rmse 

В нашем случае, информация в оригинальном виде обладает меньшей ошибкой, чем в случае стандартизации

## Использование ROC AUC вместо RMSE

Также для оценки решил изучить ROC-AUC CURVE.
Источник того, как выполнял дальнейшие действия: https://colab.research.google.com/github/dataprofessor/code/blob/master/python/ROC_curve.ipynb 

## Prediction probabilities

In [ ]:
r_probs = [0 for _ in range(len(y_test))]
knn_probs = []
for i in range(len(trainX)):
  knn_probs.append(knn.predict_proba(testX[i]))

Probabilities for the positive outcome is kept.

In [ ]:
for i in range(len(trainX)):  
  knn_probs[i] = knn_probs[i][:, 1]

In [ ]:
knn_probs[0]

## Computing AUROC and ROC curve values

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

### **Calculate AUROC**
**ROC** is the receiver operating characteristic
**AUROC** is the area under the ROC curve

In [ ]:
r_auc = roc_auc_score(y_test, r_probs)
knn_auc = []
for i in range(len(trainX)):
  knn_auc.append(roc_auc_score(y_test, knn_probs[i]))

### **Print AUROC scores**

In [ ]:
print('Random (chance) Prediction: AUROC = %.3f' % (r_auc))  
print('KNN with original data: AUROC = %.3f' % (knn_auc[0]))
print('KNN with data after normalization: AUROC = %.3f' % (knn_auc[1]))
print('KNN with data after standardization: AUROC = %.3f' % (knn_auc[2]))

### **Calculate ROC curve**

In [ ]:
r_fpr, r_tpr, _ = roc_curve(y_test, r_probs, pos_label = 2)
orig_fpr, orig_tpr, _ = roc_curve(y_test, knn_probs[0], pos_label = 2)
norm_fpr, norm_tpr, _ = roc_curve(y_test, knn_probs[1], pos_label = 2)
stand_fpr, stand_tpr, _ = roc_curve(y_test, knn_probs[2], pos_label = 2)

## **Plot the ROC curve**

In [ ]:
plt.plot(r_fpr, r_tpr, linestyle='--', label='Random prediction (AUROC = %0.3f)' % r_auc)
plt.plot(orig_fpr, orig_tpr, marker='.', label='KNN with original data: AUROC = %.3f' % knn_auc[0])
plt.plot(norm_fpr, norm_tpr, marker='.', label='KNN with data after normalization: AUROC = %.3f' % knn_auc[1])
plt.plot(stand_fpr, stand_tpr, marker='.', label='KNN with data after standardization: AUROC = %.3f' % knn_auc[2])

# Title
plt.title('ROC Plot')
# Axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# Show legend
plt.legend() # 
# Show plot
plt.show()